In [1]:
import numpy as np 
import pandas as pd

from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from pandas import DataFrame 
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization
from tensorflow.keras import Input

import seaborn as sns
import cv2
import matplotlib.pyplot as plt
import time

In [2]:
train = pd.read_csv("C:/Users/surface/OneDrive/Desktop/Fall 2023/machine learning/fashion-mnist_train.csv")
test = pd.read_csv("C:/Users/surface/OneDrive/Desktop/Fall 2023/machine learning/fashion-mnist_test.csv")

In [3]:
train.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
train.isnull().sum().sum()

0

In [5]:
test.isnull().sum().sum()

0

In [24]:
X_train = train.iloc[:,1:]
y_train = train.iloc[:,0]



In [25]:
X_test = test.iloc[:,1:]
y_test = test.iloc[:,0]


In [23]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping


early_stopping = EarlyStopping(monitor='val_loss',patience=10,verbose = 1,restore_best_weights=True)
X_train = np.asarray(X_train)
X_test = np.asarray(X_test)

X_train = X_train.reshape((X_train.shape[0], 28, 28, 1)).astype('float32') / 255.0
X_test = X_test.reshape((X_test.shape[0], 28, 28, 1)).astype('float32') / 255.0

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', strides=1, padding='same', data_format='channels_last'))
model.add(BatchNormalization())

model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', strides=1, padding='same', data_format='channels_last'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', strides=1, padding='same', data_format='channels_last'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
    
    
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', strides=1, padding='same', data_format='channels_last'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Data augmentation using ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=20,
    vertical_flip=True,
)

# Fit the model with augmented data
datagen.fit(X_train)

# Choose the number of epochs and steps per epoch
epochs = 200

# Train the model with augmented data
model.fit(X_train, y_train, batch_size=128, epochs=epochs, validation_data=(X_val, y_val), callbacks=[early_stopping])

# Evaluate the model on the test set
accuracy = model.evaluate(X_test, y_test, verbose=0)[1]
print(f'Test Accuracy: {accuracy * 100:.2f}%')

Epoch 1/200
375/375 [==============================] - 225s 588ms/step - loss: 0.5372 - accuracy: 0.8137 - val_loss: 1.7591 - val_accuracy: 0.4357
Epoch 2/200
375/375 [==============================] - 214s 572ms/step - loss: 0.3377 - accuracy: 0.8791 - val_loss: 0.2827 - val_accuracy: 0.9018
Epoch 3/200
375/375 [==============================] - 214s 571ms/step - loss: 0.2800 - accuracy: 0.9009 - val_loss: 0.2474 - val_accuracy: 0.9104
Epoch 4/200
375/375 [==============================] - 200s 534ms/step - loss: 0.2544 - accuracy: 0.9108 - val_loss: 0.2559 - val_accuracy: 0.9070
Epoch 5/200
375/375 [==============================] - 194s 517ms/step - loss: 0.2278 - accuracy: 0.9189 - val_loss: 0.2491 - val_accuracy: 0.9119
Epoch 6/200
375/375 [==============================] - 188s 501ms/step - loss: 0.2033 - accuracy: 0.9260 - val_loss: 0.2094 - val_accuracy: 0.9260
Epoch 7/200
375/375 [==============================] - 220s 587ms/step - loss: 0.1907 - accuracy: 0.9320 - val_loss: 0

In [26]:
X_train = np.asarray(X_train)
X_test = np.asarray(X_test)

X_train = X_train.reshape((X_train.shape[0], 28, 28, 1)).astype('float32') / 255.0
X_test = X_test.reshape((X_test.shape[0], 28, 28, 1)).astype('float32') / 255.0

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

model1 = Sequential()
model1.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', strides=1, padding='same', data_format='channels_last'))
model1.add(BatchNormalization())

model1.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', strides=1, padding='same', data_format='channels_last'))
model1.add(BatchNormalization())
model1.add(Dropout(0.25))

model1.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', strides=1, padding='same', data_format='channels_last'))
model1.add(MaxPooling2D(pool_size=(2, 2)))
model1.add(Dropout(0.25))
    
    
model1.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', strides=1, padding='same', data_format='channels_last'))
model1.add(BatchNormalization())
model1.add(Dropout(0.25))

model1.add(Flatten())
model1.add(Dense(512, activation='relu'))
model1.add(BatchNormalization())
model1.add(Dropout(0.5))
model1.add(Dense(128, activation='relu'))
model1.add(BatchNormalization())
model1.add(Dropout(0.5))
model1.add(Dense(10, activation='softmax'))

model1.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=128, epochs=50, validation_data=(X_val, y_val))

accuracy1 = model.evaluate(X_test, y_test, verbose=0)[1]
print(f'Test Accuracy: {accuracy1 * 100:.2f}%')


Epoch 1/50
375/375 [==============================] - 229s 605ms/step - loss: 0.1429 - accuracy: 0.9493 - val_loss: 0.2003 - val_accuracy: 0.9322
Epoch 2/50
375/375 [==============================] - 214s 571ms/step - loss: 0.1293 - accuracy: 0.9533 - val_loss: 0.2162 - val_accuracy: 0.9279
Epoch 3/50
375/375 [==============================] - 191s 509ms/step - loss: 0.1133 - accuracy: 0.9588 - val_loss: 0.1929 - val_accuracy: 0.9388
Epoch 4/50
375/375 [==============================] - 198s 528ms/step - loss: 0.1037 - accuracy: 0.9633 - val_loss: 0.1974 - val_accuracy: 0.9355
Epoch 5/50
375/375 [==============================] - 227s 605ms/step - loss: 0.0952 - accuracy: 0.9659 - val_loss: 0.2046 - val_accuracy: 0.9388
Epoch 6/50
375/375 [==============================] - 259s 692ms/step - loss: 0.0864 - accuracy: 0.9686 - val_loss: 0.1987 - val_accuracy: 0.9387
Epoch 7/50
375/375 [==============================] - 246s 655ms/step - loss: 0.0823 - accuracy: 0.9711 - val_loss: 0.2159 -